<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/cohesion_test/%5Bdistiluse_base_multilingual_cased_v1%5Dsentence_transformer_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# sentence transformers 로딩

In [149]:
pip install -U sentence-transformers datasets

In [150]:
!pip install -q Korpora

In [151]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from Korpora import Korpora

# Korpora STS (korsts)
https://github.com/kakaobrain/KorNLUDatasets

In [152]:
Korpora.fetch('korsts',root_dir='.')

[Korpora] Corpus `korsts` is already installed at /content/korsts/sts-train.tsv
[Korpora] Corpus `korsts` is already installed at /content/korsts/sts-dev.tsv
[Korpora] Corpus `korsts` is already installed at /content/korsts/sts-test.tsv


In [153]:
datasets = pd.read_csv('korsts/sts-train.tsv', sep='\t', error_bad_lines=False)

b'Skipping line 2096: expected 7 fields, saw 8\nSkipping line 2511: expected 7 fields, saw 8\nSkipping line 2652: expected 7 fields, saw 8\nSkipping line 2729: expected 7 fields, saw 8\nSkipping line 2818: expected 7 fields, saw 8\nSkipping line 3063: expected 7 fields, saw 8\nSkipping line 3385: expected 7 fields, saw 8\n'


In [154]:
datasets.head()

,genre,filename,year,id,score,sentence1,sentence2
0,main-captions,MSRvid,2012test,1,5.00,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.
1,main-captions,MSRvid,2012test,4,3.80,한 남자가 큰 플루트를 연주하고 있다.,남자가 플루트를 연주하고 있다.
2,main-captions,MSRvid,2012test,5,3.80,한 남자가 피자에 치즈를 뿌려놓고 있다.,한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.
3,main-captions,MSRvid,2012test,6,2.60,세 남자가 체스를 하고 있다.,두 남자가 체스를 하고 있다.
4,main-captions,MSRvid,2012test,9,4.25,한 남자가 첼로를 연주하고 있다.,자리에 앉은 남자가 첼로를 연주하고 있다.


In [155]:
datasets['score_sim'] = -1
datasets['score_sim'] = round(datasets['score']/5, 2)

In [156]:
train_df = datasets

In [157]:
import numpy as np

train_df['cos_sim'] = np.nan # 컬럼 생성 및 초기화
train_df['abs_diff'] = np.nan # 컬럼 생성 및 초기화

In [158]:
train_df.head()

,genre,filename,year,id,score,sentence1,sentence2,score_sim,cos_sim,abs_diff
0,main-captions,MSRvid,2012test,1,5.00,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.,1.00,NaN,NaN
1,main-captions,MSRvid,2012test,4,3.80,한 남자가 큰 플루트를 연주하고 있다.,남자가 플루트를 연주하고 있다.,0.76,NaN,NaN
2,main-captions,MSRvid,2012test,5,3.80,한 남자가 피자에 치즈를 뿌려놓고 있다.,한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.,0.76,NaN,NaN
3,main-captions,MSRvid,2012test,6,2.60,세 남자가 체스를 하고 있다.,두 남자가 체스를 하고 있다.,0.52,NaN,NaN
4,main-captions,MSRvid,2012test,9,4.25,한 남자가 첼로를 연주하고 있다.,자리에 앉은 남자가 첼로를 연주하고 있다.,0.85,NaN,NaN


# 모델 로드

In [159]:
import torch
from sentence_transformers import SentenceTransformer, util

model_name = 'sentence-transformers/distiluse-base-multilingual-cased-v1'


embedding_model = models.Transformer(model_name)

pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

model = SentenceTransformer(modules=[embedding_model, pooler])


# 문장 유사도 평가 및 차이 추출

In [160]:
s1 = 'sentence1'
s2 = 'sentence2'

for i, row in train_df[:1000].iterrows():
  emb1 = model.encode(row[s1])
  emb2 = model.encode(row[s2])
  cos_score = abs(util.pytorch_cos_sim(emb1, emb2)[0])

  train_df['cos_sim'][i] = round(cos_score.item(), 2)
  train_df['abs_diff'][i] = abs(row['score_sim'] - round(cos_score.item(), 2))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [161]:
train_df[:20]

,genre,filename,year,id,score,sentence1,sentence2,score_sim,cos_sim,abs_diff
0,main-captions,MSRvid,2012test,1,5.000,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.,1.00,1.00,0.00
1,main-captions,MSRvid,2012test,4,3.800,한 남자가 큰 플루트를 연주하고 있다.,남자가 플루트를 연주하고 있다.,0.76,0.89,0.13
2,main-captions,MSRvid,2012test,5,3.800,한 남자가 피자에 치즈를 뿌려놓고 있다.,한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.,0.76,0.91,0.15
3,main-captions,MSRvid,2012test,6,2.600,세 남자가 체스를 하고 있다.,두 남자가 체스를 하고 있다.,0.52,0.88,0.36
4,main-captions,MSRvid,2012test,9,4.250,한 남자가 첼로를 연주하고 있다.,자리에 앉은 남자가 첼로를 연주하고 있다.,0.85,0.86,0.01
5,main-captions,MSRvid,2012test,11,4.250,몇몇 남자들이 싸우고 있다.,두 남자가 싸우고 있다.,0.85,0.89,0.04
6,main-captions,MSRvid,2012test,12,0.500,남자가 담배를 피우고 있다.,남자가 스케이트를 타고 있다.,0.10,0.39,0.29
7,main-captions,MSRvid,2012test,13,1.600,남자가 피아노를 치고 있다.,남자가 기타를 연주하고 있다.,0.32,0.64,0.32
8,main-captions,MSRvid,2012test,14,2.200,한 남자가 기타를 치고 노래를 부르고 있다.,한 여성이 어쿠스틱 기타를 연주하고 노래를 부르고 있다.,0.44,0.78,0.34
9,main-captions,MSRvid,2012test,16,5.000,사람이 고양이를 천장에 던지고 있다.,사람이 고양이를 천장에 던진다.,1.00,0.99,0.01


# 오차 평균 값 계산

In [162]:
diff_total = train_df['abs_diff'][:1000].sum()
print(f'{model_name}의 오차 평균값 : ',diff_total/1000)

sentence-transformers/distiluse-base-multilingual-cased-v1의 오차 평균값 :  0.22211
